In [1]:
import pandas as pd 
#%pip install langchain_groq
from langchain_groq.chat_models import ChatGroq
import pandas as pd
import numpy as np
from itertools import product
from sklearn import tree
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, recall_score
import ast
import time
import matplotlib.pyplot as plt
import tsfel

In [2]:
# Groq API and Models 
Groq_Token = open('..\\groqapi.txt', 'r').read().split("\n")[0]  # Do not share this key with anyone
groq_models = {"llama3-70b": "llama3-70b-8192", "mixtral": "mixtral-8x7b-32768", "gemma-7b": "gemma-7b-it","llama3.1-70b":"llama-3.1-70b-versatile","llama3-8b":"llama3-8b-8192","llama3.1-8b":"llama-3.1-8b-instant","gemma-9b":"gemma2-9b-it"}

Let us load the UCI HAR Data

In [3]:
x_train = np.load("..\\X_train.npy")
y_train = np.load("..\\Y_train.npy")


In [4]:
print(x_train.shape)
print(y_train.shape)

(126, 500, 3)
(126,)


In [ ]:
cfg = tsfel.get_features_by_domain()
feature_df = tsfel.time_series_features_extractor(cfg, x_train, verbose = 0, fs = 50)
feature_df_test = tsfel.time_series_features_extractor(cfg, x_test, verbose = 0, fs = 50)

In [7]:
def makedataset(X, y):
    activity_dic = {1: "WALKING", 2: "WALKING_UPSTAIRS", 3: "WALKING_DOWNSTAIRS", 4: "SITTING", 5: "STANDING", 6: "LAYING"}
    flat_x = X.reshape(-1, 3)
    flat_y = np.repeat(y, 500)
    #print(flat_y)
    replaced_y = [activity_dic[i] for i in flat_y]
    df = pd.DataFrame({"activity": replaced_y, "accx": flat_x[:, 0], "accy": flat_x[:, 1], "accz": flat_x[:, 2]})
    df['acc_mag_diff'] = ((df['accx']**2+df['accy']**2+df['accz']**2)-1.08)**2
    return df  

In [8]:
train_df = makedataset(x_train, y_train)
test_df = makedataset(x_test, y_test)

,activity,accx,accy,accz,acc_mag_diff
0,WALKING_UPSTAIRS,1.196480,-0.160690,-0.740178,0.856086
1,WALKING_UPSTAIRS,1.222030,-0.267295,-0.771748,1.167264
2,WALKING_UPSTAIRS,1.223110,-0.433863,-0.767842,1.425199
3,WALKING_UPSTAIRS,1.119198,-0.533790,-0.756309,1.059952
4,WALKING_UPSTAIRS,0.915843,-0.496126,-0.629135,0.160576
...,...,...,...,...,...
62995,WALKING_UPSTAIRS,0.834664,-0.084789,-0.402108,0.045991
62996,WALKING_UPSTAIRS,0.741997,-0.107038,-0.368874,0.145860
62997,WALKING_UPSTAIRS,0.652846,-0.106747,-0.345863,0.273294
62998,WALKING_UPSTAIRS,0.640395,-0.106293,-0.339603,0.295137
